# Preparatory steps:
 - Import python packages.
 - Set your working directory to the location of the data files.
 - Read in the data files.

In [2]:
# data processing
import numpy as np
import pandas as pd
from datetime import date
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", 999)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

## Set LOCAL_REPOSITORY_LOCATION/DAPT/costar/data_files as your working directory

**Note:** This approach works for members of the team who have downloaded the raw files to a filepath like this one: <br>
 <ul><li><em>C:\Users\username\Documents\LOCAL_REPOSITORY_LOCATION\DAPT\costar\data_files</em></li></ul><br>
If you have these files saved elsewhere, please update the path. 

In [3]:
# repos (environment variables set)
import os

LOCAL_REPOSITORY_LOCATION = os.environ.get('LOCAL_REPOSITORY_LOCATION')

# also note: you should replace "short" with your username!
os.chdir(r'C:\Users\short\Documents\LOCAL_REPOSITORY_LOCATION\DAPT\costar\data_files')

print(os.getcwdb())

b'C:\\Users\\short\\Documents\\LOCAL_REPOSITORY_LOCATION\\DAPT\\costar\\data_files'


## Read in the raw data files:
- **costar_0621.xlsx**
  - Renewal
  - Days on Market
  - Rental
- **EstimatedRentAndGeography.xlsx**

In [4]:
renewal = pd.read_excel(r'costar_0621.xlsx', sheet_name = 0)

In [5]:
days_on_market = pd.read_excel(r'costar_0621.xlsx', sheet_name = 1)

In [6]:
rental = pd.read_excel(r'costar_0621.xlsx', sheet_name = 2)

In [7]:
estimated_rent_geography = pd.read_excel(r'EstimatedRentAndGeography.xlsx', sheet_name = 0)

In [8]:
lease_data = pd.read_csv(r'LeaseDataOut.csv', encoding = 'ISO-8859-2', keep_default_na = True)

## Take a look at the number of rows/columns in each file.

In [9]:
print('renewal shape: ', renewal.shape)
print('days on market shape: ', days_on_market.shape)
print('rental shape: ', rental.shape)
print('estimated_rent_geography: ',estimated_rent_geography.shape)
print('lease_data: ',lease_data.shape)

renewal shape:  (215536, 2)
days on market shape:  (595273, 7)
rental shape:  (217170, 2)
estimated_rent_geography:  (929262, 5)
lease_data:  (1057504, 19)


# **renewal**

## Overview of **renewal**

The **renewal** data frame describes whether there was a **Renewal** for the provided **LeaseDealID**s. 
 - **LeaseDealID**: A property can have multiple leases for space within the building. Leases can span multiple floors. 
 - **Renewal**: True (1) or False (0). Looks at what we know about the occupancy of the building immediately preceding the lease. The main assumption is that if the space was never vacant and never advertised then it was a renewal.

In [68]:
renewal.head(3)

,LeaseDealID,Renewal
0,30037095,0
1,112633161,0
2,110461287,0


In [75]:
renewal.describe()

,LeaseDealID,Renewal
count,215536.0,215536.0
mean,81238250.8,0.7
std,44526382.9,0.4
min,10000015.0,0.0
25%,30076767.2,0.0
50%,110599151.0,1.0
75%,112307845.2,1.0
max,189174581.0,1.0


## First, we check for duplicate **LeaseDealID**s.

Of 215,536 rows, there are 215,536 unique values for LeaseDealID, so we do not need to resolve duplicate values.

In [79]:
renewal.nunique(axis = 0)

LeaseDealID    215536
Renewal             2
dtype: int64

# **days_on_market**

## Overview of **days_on_market**

**days_on_market** data frame provides us with the following attributes for the provided **LeaseDealID**s:
 - **SqFtMin**: Tenant's rentable area
 - **SqFtMax**: Tenant's rentable area + common space (e.g. closets, restrooms, mezzanines)
 - **DateOnMarket**: date space was on market
 - **DateOffMarket**: date space was off market
 - **DaysOnMarket**: Represents the amount of time a space was vacant before someone moved in. We call this the ‘downtime’, and because it’s fundamental to vacancy, it’s probably the most important assumption in a DCF model that we can do a really good job capturing.
 - **ActualVacancy**: When we have a non-null space available on date and a non-null commencement date, which means that we knew the space was going to become available and space was empty afterwards.

We notice that values in **DaysOnMarket** seem a little low. After confirming with Rob, we decide to impute this manually instead.

In [81]:
days_on_market.head(3)

,LeaseDealID,SqFtMin,SqFtMax,DateOnMarket,DateOffMarket,DaysOnMarket,ActualVacancy
0,160018621,1249,1249,2018-03-31,2018-12-31,256,1
1,112570898,1831,2177,2011-12-31,2012-03-31,61,1
2,112683000,1831,1831,2011-12-31,2012-06-30,152,1


In [85]:
# Check the unique values in each column
days_on_market.nunique(axis = 0)

LeaseDealID      585502
SqFtMin           22146
SqFtMax           24246
DateOnMarket        120
DateOffMarket        83
ActualVacancy         2
DaysOnMarket        252
dtype: int64

## Drop the column **DaysOnMarket** which was provided originally.

In [82]:
del days_on_market['DaysOnMarket']

## Impute the **DaysOnMarket** attribute from the difference between **DateOnMarket** and **DateOffMarket**.

In [83]:
# re-create the column
days_on_market['DaysOnMarket'] = (days_on_market['DateOffMarket'] - days_on_market['DateOnMarket']).dt.days

# Rearrange the columns
days_on_market = days_on_market[['LeaseDealID', 'SqFtMin', 'SqFtMax', 'DateOnMarket', 'DateOffMarket', 'DaysOnMarket', 'ActualVacancy']]

## Looking at the output again, this looks more accurate.

In [87]:
days_on_market.head(3)

,LeaseDealID,SqFtMin,SqFtMax,DateOnMarket,DateOffMarket,DaysOnMarket,ActualVacancy
0,160018621,1249,1249,2018-03-31,2018-12-31,275,1
1,112570898,1831,2177,2011-12-31,2012-03-31,91,1
2,112683000,1831,1831,2011-12-31,2012-06-30,182,1


## Next, we'll check for duplicates by column.

We see that this file contains more than twice as many LeaseDealIDs (585,502) compared against the **renewal** file (215,536).

## We also note that this file has 9,771 duplicate LeaseDealIDs.

In [89]:
# Checking if there are any duplicate LeaseDealIDs in days_on_market
days_on_market.LeaseDealID.duplicated().sum()

9771

## We want to understand those duplicates better, so we'll take a look.

In [16]:
#Checking the duplicated rows
days_on_market[days_on_market.LeaseDealID.duplicated()]

,LeaseDealID,SqFtMin,SqFtMax,DateOnMarket,DateOffMarket,DaysOnMarket,ActualVacancy
83,122263421,1617,1617,2014-06-30,2016-03-31,640,0
87,112330867,2000,2000,2010-12-31,2011-03-31,90,1
184,165994791,2500,2500,2018-12-31,2019-09-30,273,1
185,165994791,1500,1500,2018-12-31,2019-09-30,273,1
187,113315805,985,985,2011-12-31,2012-09-30,274,1
...,...,...,...,...,...,...,...
594897,113300851,1151,1151,2011-12-31,2012-09-30,274,1
594986,112347775,4697,4697,2010-12-31,2011-06-30,181,1
595121,112155039,19870,19870,2010-09-30,2010-12-31,92,0
595200,112445545,5305,5305,2010-09-30,2011-12-31,457,0


## 9,771 rows is a lot to sift through, so we'll look at a few examples, starting with **LeaseDealID** `112347775`.

We can see that these have the same LeaseDealID, but different **DateOnMarket**, **SqFtMin**, and **SqFtMax** values. 

In [91]:
# Cheking what the difference in the LeaseDealIDs that are repeated
days_on_market[days_on_market.LeaseDealID == 112347775]

,LeaseDealID,SqFtMin,SqFtMax,DateOnMarket,DateOffMarket,DaysOnMarket,ActualVacancy
594985,112347775,2399,2399,2010-09-30,2011-06-30,273,1
594986,112347775,4697,4697,2010-12-31,2011-06-30,181,1


# **rental**

## Overview of **rental**

We see that this is a two-column file providing us with the **EstimatedRent** for the given **LeaseDealID**s.
 - **Estimated_Rent**: Estimate of what the rent in the building should have been when the lease was signed. One important distinction is that it’s a gross rent, so if a lease was signed with some service type other than gross it probably isn’t very comparable.

Brie's questions for team: 
 - Is this is a monthly rental rate?
 - Should we convert this value (e.g., multiple by square footage)?

In [93]:
#View few rows of rental
rental.head(3)

,LeaseDealID,EstimatedRent
0,112403492,39.8
1,112451500,36.5
2,111489470,35.6


## We check for duplicate values in **rental**.

We see that this file contains a similar amount of **LeaseDealID**s (217,170) to the **renewal** file (215,536).

In [13]:
# Check the unique values in each column
unique_rental = rental.nunique(axis = 0)

unique_rental.reset_index()

print(unique_rental)

LeaseDealID      217170
EstimatedRent    184861
dtype: int64


## After checking for NULL values and finding none, we see that there are 32,309 **LeaseDealID**s with non-unique rental rates.

In [14]:
rental.LeaseDealID.isna().sum()

0

In [15]:
# LeaseDealIDs with a non-unique EstimatedRent value
(unique_rental['LeaseDealID'] - unique_rental['EstimatedRent'])

32309

# **estimated_rent_geography**

## Overview of **estimated_rent_geography**

The **estimated_rent_geography** dataframe provides us with the following attributes for the given **LeaseDealID**s:
 - **estimatedrent**: Estimate of what the rent in the building should have been when the lease was signed. One important distinction is that it’s a gross rent, so if a lease was signed with some service type other than gross it probably isn’t very comparable.
 - **researchmarketname**: provides name of closest major metropolitan area
 - **submarketname**: like "neighborhood" (e.g. "Innsbrook" is in a neighborhood in Richmond)
 -  **zip**: The corresponding zipcode for a given **LeaseDealID**.

In [22]:
estimated_rent_geography.head(3)

,LeaseDealID,estimatedrent,researchmarketname,submarketname,zip
0,124461891,15.055131,Other Market Areas,NaN,42701
1,111152193,14.356337,Other Market Areas,NaN,59102


In [112]:
len(pd.unique(estimated_rent_geography['LeaseDealID']))

929262

# **lease_data**

## Overview of **lease_data**

In [11]:
lease_data.head(3)

,Unnamed: 0,PropertyID,LeaseDealID,LeaseSignDate,FromDate,LeaseExpirationDate,ToDate,RateActual,ServiceTypeID,PropertyTypeID,LocationOccupancyID,RBA,TenantImprovementAllowancePerSqFt,FreeMonths,LeaseTermInMonths,CBSAID,CBSADesc,ConstructionYear,BuildingRatingID
0,1,157648.0,110498312,2006-09-05,2006-10-05,2011-10-05,2014-04-29,NaN,4.0,5,111764312.0,3650.0,NaN,NaN,60.0,35614.0,"New York-Jersey City-White Plains, NY-NJ",1928.0,3
1,2,76048.0,30028220,1981-12-16,1982-01-15,2000-04-14,2001-04-15,NaN,11.0,5,30028220.0,NaN,NaN,NaN,219.0,28140.0,"Kansas City, MO-KS",1977.0,3
2,3,559441.0,30187227,1985-12-16,1986-01-15,2000-03-14,2000-03-14,NaN,NaN,5,30187227.0,NaN,NaN,NaN,170.0,28140.0,"Kansas City, MO-KS",1985.0,2


In [12]:
lease_data.describe()

,Unnamed: 0,PropertyID,LeaseDealID,RateActual,ServiceTypeID,PropertyTypeID,LocationOccupancyID,RBA,TenantImprovementAllowancePerSqFt,FreeMonths,LeaseTermInMonths,CBSAID,ConstructionYear,BuildingRatingID
count,1057504.0,1057504.0,1057504.0,216946.0,823308.0,1057504.0,1057504.0,1032709.0,22352.0,138125.0,1057504.0,1055400.0,1040150.0,1057504.0
mean,528752.5,1269296.9,91048286.1,22.4,8.3,5.0,99989529.0,5271.1,11.1,1.6,48.1,29447.9,1973.9,2.9
std,305275.3,2234722.1,40956979.7,352.8,2.8,0.0,53087445.3,17068.5,191.6,1.8,39.9,11417.2,27.0,0.8
min,1.0,1.0,10000015.0,0.0,1.0,5.0,10000015.0,0.0,-130.0,0.0,-395.0,10100.0,1719.0,1.0
25%,264376.8,178605.0,70094623.8,12.0,7.0,5.0,70096857.5,1011.0,0.0,1.0,24.0,19100.0,1970.0,2.0
50%,528752.5,381601.0,111323092.0,16.2,7.0,5.0,114074003.0,2000.0,0.0,1.0,36.0,31084.0,1982.0,3.0
75%,793128.2,795031.0,113503979.5,22.0,10.0,5.0,121605313.8,4153.0,0.0,2.0,60.0,38300.0,1988.0,3.0
max,1057504.0,11605746.0,187388961.0,84000.0,19.0,5.0,284204561.0,2000000.0,9355.0,185.0,1236.0,49820.0,2022.0,5.0


# Prepare to merge the original **lease_data** with these newer data sets.

## Consider the total number of **LeaseDealID**s across all five data sets.

The **lease_data** is our largest, followed by **estimated_rent_geography**: <br>
| Data frame               | Row count |
| ---                      | ---       |
| lease_data               | 1,057,504 |
| estimated_rent_geography | 929,262   |
| days on market           | 595,273   |
| rental                   |  217,170  |
| renewal                  | 215,536   |



## We merge the **renewal** and **days_on_market** dataframes, using a left join.

In [64]:
#Merging the datasets renewal and days_on_market

lease = pd.merge(renewal,
                 days_on_market,
                 on='LeaseDealID', 
                 how='outer')
lease.shape

(727673, 8)

In [110]:
lease.head(3)

,PropertyID,LeaseDealID,LeaseSignDate,FromDate,LeaseExpirationDate,ToDate,RateActual,ServiceTypeID,PropertyTypeID,LocationOccupancyID,RBA,TenantImprovementAllowancePerSqFt,FreeMonths,LeaseTermInMonths,CBSAID,CBSADesc,ConstructionYear,BuildingRatingID,Renewal,SqFtMin,SqFtMax,DateOnMarket,DateOffMarket,DaysOnMarket,ActualVacancy
0,157648,110498312,2006-09-05,2006-10-05,2011-10-05,2014-04-29,NaN,4.0,5,111764312,3650.0,NaN,NaN,60.0,35614.0,"New York-Jersey City-White Plains, NY-NJ",1928.0,3,NaN,3650.0,3650.0,2006-03-31,2006-12-31,275.0,1.0
1,76048,30028220,1981-12-16,1982-01-15,2000-04-14,2001-04-15,NaN,11.0,5,30028220,NaN,NaN,NaN,219.0,28140.0,"Kansas City, MO-KS",1977.0,3,NaN,NaN,NaN,NaT,NaT,NaN,NaN


## We merge the original **lease_data** into the newly created **lease** dataframe, using a left join.

In [111]:
#now merging with original Lease data (Did a left join because we need all the leasedeal ID's that have property_Id's)
lease = pd.merge(LeaseData,
                 lease,
                 on='LeaseDealID', 
                 how='left')
lease.shape

(1067141, 42)

In [68]:
lease.head(3)

,PropertyID,LeaseDealID,LeaseSignDate,FromDate,LeaseExpirationDate,ToDate,RateActual,ServiceTypeID,PropertyTypeID,LocationOccupancyID,RBA,TenantImprovementAllowancePerSqFt,FreeMonths,LeaseTermInMonths,CBSAID,CBSADesc,ConstructionYear,BuildingRatingID,Renewal,SqFtMin,SqFtMax,DateOnMarket,DateOffMarket,DaysOnMarket,ActualVacancy
0,157648,110498312,2006-09-05,2006-10-05,2011-10-05,2014-04-29,NaN,4.0,5,111764312,3650.0,NaN,NaN,60.0,35614.0,"New York-Jersey City-White Plains, NY-NJ",1928.0,3,NaN,3650.0,3650.0,2006-03-31,2006-12-31,275.0,1.0
1,76048,30028220,1981-12-16,1982-01-15,2000-04-14,2001-04-15,NaN,11.0,5,30028220,NaN,NaN,NaN,219.0,28140.0,"Kansas City, MO-KS",1977.0,3,NaN,NaN,NaN,NaT,NaT,NaN,NaN


In [69]:
print('Lease full file shape: ', lease.shape)

Lease full file shape:  (1067141, 25)


In [70]:
#Check unique leasedeal_id in the merged lease dataframe excluding rental and estimated_rent dataframes
len(pd.unique(lease['LeaseDealID']))

1057504

In [72]:
#Checking if there are any null Property_Id's in merged lease dataframe
lease[lease['PropertyID'].isnull()]

,PropertyID,LeaseDealID,LeaseSignDate,FromDate,LeaseExpirationDate,ToDate,RateActual,ServiceTypeID,PropertyTypeID,LocationOccupancyID,RBA,TenantImprovementAllowancePerSqFt,FreeMonths,LeaseTermInMonths,CBSAID,CBSADesc,ConstructionYear,BuildingRatingID,Renewal,SqFtMin,SqFtMax,DateOnMarket,DateOffMarket,DaysOnMarket,ActualVacancy


In [73]:
#Merge rental and estimated rent files to use rental eatimated rent column to fill in missing values in Estimated_geog
est_rent = pd.merge(rental, estimated_rent_geography, on=['LeaseDealID'], how='outer')
est_rent.head(3)

,LeaseDealID,EstimatedRent,estimatedrent,researchmarketname,submarketname,zip
0,112403492,39.774563,NaN,NaN,NaN,NaN
1,112451500,36.519464,36.006552,"Washington, DC",Capitol Hill,20002.0
2,111489470,35.613787,34.847497,"Washington, DC",CBD,20036.0
3,110142376,32.260847,31.513739,"Washington, DC",CBD,20036.0
4,112399795,40.037640,40.218861,"Washington, DC",CBD,20037.0


In [74]:
#Check how many leasedeal_id's match in Rental and Estimated_rent_geography
estimated_rent_geography['LeaseDealID'].isin(rental['LeaseDealID']).value_counts()

False    718978
True     210284
Name: LeaseDealID, dtype: int64

#### REPLACING ESTIMATED_RENT_GEOGRAPHY ESTIMATED RENT WITH ESTIMATED RENT FROM RENTAL WHEREVER THERE ARE MISSING VALUES OF ESTIMATED RENT IN ESTIMATED_RENT_GEOGRAPHY DATA

In [75]:
#where ever there are Nan values in estimated rent, replace Nan's with EstimatedRent values
est_rent.estimatedrent.fillna(est_rent.EstimatedRent, inplace=True)

In [76]:
est_rent[pd.isnull(est_rent['estimatedrent'])]
est_rent.shape

(936148, 6)

In [77]:
#Number of leasedeal ID's that match in lease and est_rent dataframe
lease['LeaseDealID'].isin(est_rent['LeaseDealID']).value_counts()

True     928207
False    138934
Name: LeaseDealID, dtype: int64

In [78]:
#Check the unique values in each column of est_rent
est_rent.nunique(axis=0)

LeaseDealID           936148
EstimatedRent         184861
estimatedrent         684740
researchmarketname       139
submarketname           2557
zip                    11716
dtype: int64

In [79]:
est_rent.head(3)

,LeaseDealID,EstimatedRent,estimatedrent,researchmarketname,submarketname,zip
0,112403492,39.774563,39.774563,NaN,NaN,NaN
1,112451500,36.519464,36.006552,"Washington, DC",Capitol Hill,20002.0


In [80]:
#Check how many null values are there in estimatedrent column
#only 1 which is good
est_rent['estimatedrent'].isnull().values.sum()

1

In [81]:
#See that one column which is null
est_rent[est_rent['estimatedrent'].isnull()]

,LeaseDealID,EstimatedRent,estimatedrent,researchmarketname,submarketname,zip
223900,111334881,NaN,NaN,Other Market Areas,NaN,78076.0


In [82]:
#Drop rental EstimatedRent column--we can drop it since we have merged and used all from rantal dataframe
del est_rent['EstimatedRent']

In [83]:
est_rent.head(3)

,LeaseDealID,estimatedrent,researchmarketname,submarketname,zip
0,112403492,39.774563,NaN,NaN,NaN
1,112451500,36.006552,"Washington, DC",Capitol Hill,20002.0
2,111489470,34.847497,"Washington, DC",CBD,20036.0
3,110142376,31.513739,"Washington, DC",CBD,20036.0
4,112399795,40.218861,"Washington, DC",CBD,20037.0


In [84]:
#Check how many leasedeal_id's match in Rental and Estimated_rent_geography
lease['LeaseDealID'].isin(est_rent['LeaseDealID']).value_counts()

True     928207
False    138934
Name: LeaseDealID, dtype: int64

In [85]:
#now merge lease dataframe with est_rent dataframe
lease_merged = pd.merge(lease,
                 est_rent,
                 on='LeaseDealID', 
                 how='left')

In [86]:
lease_merged.shape

(1067141, 29)

In [87]:
# rename Pandas columns to lower case
lease_merged.columns= lease_merged.columns.str.lower()

In [88]:
lease_merged.head(3)

,propertyid,leasedealid,leasesigndate,fromdate,leaseexpirationdate,todate,rateactual,servicetypeid,propertytypeid,locationoccupancyid,rba,tenantimprovementallowancepersqft,freemonths,leaseterminmonths,cbsaid,cbsadesc,constructionyear,buildingratingid,renewal,sqftmin,sqftmax,dateonmarket,dateoffmarket,daysonmarket,actualvacancy,estimatedrent,researchmarketname,submarketname,zip
0,157648,110498312,2006-09-05,2006-10-05,2011-10-05,2014-04-29,NaN,4.0,5,111764312,3650.0,NaN,NaN,60.0,35614.0,"New York-Jersey City-White Plains, NY-NJ",1928.0,3,NaN,3650.0,3650.0,2006-03-31,2006-12-31,275.0,1.0,39.202936,New York City,Murray Hill,10016.0
1,76048,30028220,1981-12-16,1982-01-15,2000-04-14,2001-04-15,NaN,11.0,5,30028220,NaN,NaN,NaN,219.0,28140.0,"Kansas City, MO-KS",1977.0,3,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
lease_merged.columns

Index(['propertyid', 'leasedealid', 'leasesigndate', 'fromdate',
       'leaseexpirationdate', 'todate', 'rateactual', 'servicetypeid',
       'propertytypeid', 'locationoccupancyid', 'rba',
       'tenantimprovementallowancepersqft', 'freemonths', 'leaseterminmonths',
       'cbsaid', 'cbsadesc', 'constructionyear', 'buildingratingid', 'renewal',
       'sqftmin', 'sqftmax', 'dateonmarket', 'dateoffmarket', 'daysonmarket',
       'actualvacancy', 'estimatedrent', 'researchmarketname', 'submarketname',
       'zip'],
      dtype='object')

In [90]:
#Rearrange the columns
lease_merged = lease_merged[['leasedealid', 'propertyid', 'renewal', 'sqftmin', 'sqftmax',
       'dateonmarket', 'dateoffmarket', 'daysonmarket', 'actualvacancy',
        'fromdate', 'todate', 'leasesigndate', 'leaseexpirationdate', 'leaseterminmonths',
       'rateactual', 'estimatedrent', 'servicetypeid', 'propertytypeid',
       'locationoccupancyid','rba', 'tenantimprovementallowancepersqft',
        'freemonths', 'cbsaid', 'cbsadesc', 'constructionyear',
       'buildingratingid', 'researchmarketname',
       'submarketname', 'zip']]


In [91]:
#renaming column names
lease_merged.columns = ['leasedeal_id', 'property_id', 'renewal', 'sqft_min', 'sqft_max',
       'date_on_market', 'date_off_market', 'days_on_market', 'actual_vacancy',
        'from_date', 'to_date', 'lease_sign_date', 'lease_expiration_date', 'lease_term_inmonths',
       'rate_actual', 'estimated_rent', 'service_type_id', 'property_type_id',
       'location_occupancy_id','rba', 'tenantimprovementallowancepersqft',
        'free_months', 'cbsaid', 'cbsadesc', 'construction_year',
       'buildingrating_id', 'researchmarket_name',
       'submarket_name', 'zip']

In [92]:
lease_merged.head(5)

,leasedeal_id,property_id,renewal,sqft_min,sqft_max,date_on_market,date_off_market,days_on_market,actual_vacancy,from_date,to_date,lease_sign_date,lease_expiration_date,lease_term_inmonths,rate_actual,estimated_rent,service_type_id,property_type_id,location_occupancy_id,rba,tenantimprovementallowancepersqft,free_months,cbsaid,cbsadesc,construction_year,buildingrating_id,researchmarket_name,submarket_name,zip
0,110498312,157648,NaN,3650.0,3650.0,2006-03-31,2006-12-31,275.0,1.0,2006-10-05,2014-04-29,2006-09-05,2011-10-05,60.0,NaN,39.202936,4.0,5,111764312,3650.0,NaN,NaN,35614.0,"New York-Jersey City-White Plains, NY-NJ",1928.0,3,New York City,Murray Hill,10016.0
1,30028220,76048,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1982-01-15,2001-04-15,1981-12-16,2000-04-14,219.0,NaN,NaN,11.0,5,30028220,NaN,NaN,NaN,28140.0,"Kansas City, MO-KS",1977.0,3,NaN,NaN,NaN
2,30187227,559441,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1986-01-15,2000-03-14,1985-12-16,2000-03-14,170.0,NaN,NaN,NaN,5,30187227,NaN,NaN,NaN,28140.0,"Kansas City, MO-KS",1985.0,2,NaN,NaN,NaN
3,114096677,239837,NaN,1388.0,1388.0,2013-12-31,2014-03-31,90.0,1.0,2014-02-14,2015-02-12,2014-01-15,2015-02-13,12.0,NaN,18.914058,7.0,5,122849533,1388.0,NaN,NaN,26420.0,"Houston-The Woodlands-Sugar Land, TX",1972.0,2,Houston,Riverway,77024.0
4,10587417,220914,NaN,NaN,NaN,NaT,NaT,NaN,NaN,2001-06-30,NaN,2001-05-01,2006-06-30,60.0,NaN,21.278033,NaN,5,10587417,7200.0,NaN,NaN,14460.0,"Boston-Cambridge-Newton, MA-NH",1906.0,2,Boston,Route 3 South,2382.0


In [93]:
lease_merged['estimated_rent'].isnull().values.sum()

138935

In [94]:
#Split cbsadesc into two columns splitting into cities and states
lease_merged[['cbsa_cities','cbsa_states']] = lease_merged.cbsadesc.str.split(",",expand=True)

In [95]:
#drop cbsadesc colum
del lease_merged['cbsadesc']

In [96]:
lease_merged.head(3)

,leasedeal_id,property_id,renewal,sqft_min,sqft_max,date_on_market,date_off_market,days_on_market,actual_vacancy,from_date,to_date,lease_sign_date,lease_expiration_date,lease_term_inmonths,rate_actual,estimated_rent,service_type_id,property_type_id,location_occupancy_id,rba,tenantimprovementallowancepersqft,free_months,cbsaid,construction_year,buildingrating_id,researchmarket_name,submarket_name,zip,cbsa_cities,cbsa_states
0,110498312,157648,NaN,3650.0,3650.0,2006-03-31,2006-12-31,275.0,1.0,2006-10-05,2014-04-29,2006-09-05,2011-10-05,60.0,NaN,39.202936,4.0,5,111764312,3650.0,NaN,NaN,35614.0,1928.0,3,New York City,Murray Hill,10016.0,New York-Jersey City-White Plains,NY-NJ
1,30028220,76048,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1982-01-15,2001-04-15,1981-12-16,2000-04-14,219.0,NaN,NaN,11.0,5,30028220,NaN,NaN,NaN,28140.0,1977.0,3,NaN,NaN,NaN,Kansas City,MO-KS
2,30187227,559441,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1986-01-15,2000-03-14,1985-12-16,2000-03-14,170.0,NaN,NaN,NaN,5,30187227,NaN,NaN,NaN,28140.0,1985.0,2,NaN,NaN,NaN,Kansas City,MO-KS
3,114096677,239837,NaN,1388.0,1388.0,2013-12-31,2014-03-31,90.0,1.0,2014-02-14,2015-02-12,2014-01-15,2015-02-13,12.0,NaN,18.914058,7.0,5,122849533,1388.0,NaN,NaN,26420.0,1972.0,2,Houston,Riverway,77024.0,Houston-The Woodlands-Sugar Land,TX
4,10587417,220914,NaN,NaN,NaN,NaT,NaT,NaN,NaN,2001-06-30,NaN,2001-05-01,2006-06-30,60.0,NaN,21.278033,NaN,5,10587417,7200.0,NaN,NaN,14460.0,1906.0,2,Boston,Route 3 South,2382.0,Boston-Cambridge-Newton,MA-NH


In [97]:
lease_merged.columns

Index(['leasedeal_id', 'property_id', 'renewal', 'sqft_min', 'sqft_max',
       'date_on_market', 'date_off_market', 'days_on_market', 'actual_vacancy',
       'from_date', 'to_date', 'lease_sign_date', 'lease_expiration_date',
       'lease_term_inmonths', 'rate_actual', 'estimated_rent',
       'service_type_id', 'property_type_id', 'location_occupancy_id', 'rba',
       'tenantimprovementallowancepersqft', 'free_months', 'cbsaid',
       'construction_year', 'buildingrating_id', 'researchmarket_name',
       'submarket_name', 'zip', 'cbsa_cities', 'cbsa_states'],
      dtype='object')

In [98]:
#Rearrange columns again to bring cbsaid states and cities together with cbsaid
lease_merged = lease_merged[['leasedeal_id', 'property_id', 'renewal', 'sqft_min', 'sqft_max',
       'date_on_market', 'date_off_market', 'days_on_market', 'actual_vacancy',
       'from_date', 'to_date', 'lease_sign_date', 'lease_expiration_date',
       'lease_term_inmonths', 'rate_actual', 'estimated_rent',
       'service_type_id', 'property_type_id', 'location_occupancy_id', 'rba',
       'tenantimprovementallowancepersqft', 'free_months',
       'construction_year', 'buildingrating_id', 'researchmarket_name', 'cbsaid', 'cbsa_cities', 'cbsa_states',
       'submarket_name', 'zip']]

In [99]:
lease_merged.head(3)

,leasedeal_id,property_id,renewal,sqft_min,sqft_max,date_on_market,date_off_market,days_on_market,actual_vacancy,from_date,to_date,lease_sign_date,lease_expiration_date,lease_term_inmonths,rate_actual,estimated_rent,service_type_id,property_type_id,location_occupancy_id,rba,tenantimprovementallowancepersqft,free_months,construction_year,buildingrating_id,researchmarket_name,cbsaid,cbsa_cities,cbsa_states,submarket_name,zip
0,110498312,157648,NaN,3650.0,3650.0,2006-03-31,2006-12-31,275.0,1.0,2006-10-05,2014-04-29,2006-09-05,2011-10-05,60.0,NaN,39.202936,4.0,5,111764312,3650.0,NaN,NaN,1928.0,3,New York City,35614.0,New York-Jersey City-White Plains,NY-NJ,Murray Hill,10016.0
1,30028220,76048,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1982-01-15,2001-04-15,1981-12-16,2000-04-14,219.0,NaN,NaN,11.0,5,30028220,NaN,NaN,NaN,1977.0,3,NaN,28140.0,Kansas City,MO-KS,NaN,NaN


In [100]:
lease_merged.dtypes

leasedeal_id                                  int64
property_id                                   int64
renewal                                     float64
sqft_min                                    float64
sqft_max                                    float64
date_on_market                       datetime64[ns]
date_off_market                      datetime64[ns]
days_on_market                              float64
actual_vacancy                              float64
from_date                                    object
to_date                                      object
lease_sign_date                              object
lease_expiration_date                        object
lease_term_inmonths                         float64
rate_actual                                 float64
estimated_rent                              float64
service_type_id                             float64
property_type_id                              int64
location_occupancy_id                         int64
rba         

# CHECK THE STATS

In [102]:
lease_merged.describe(include=[np.number]).T

,count,mean,std,min,25%,50%,75%,max
leasedeal_id,1067141.0,9.127985e+07,4.085930e+07,1.000002e+07,7.010694e+07,1.113457e+08,1.135171e+08,1.873890e+08
property_id,1067141.0,1.271110e+06,2.235648e+06,1.000000e+00,1.787400e+05,3.816730e+05,7.981200e+05,1.160575e+07
renewal,219610.0,7.202814e-01,4.488619e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
sqft_min,589103.0,2.848921e+03,5.631973e+03,1.000000e+00,8.970000e+02,1.514000e+03,2.895000e+03,4.969840e+05
sqft_max,589103.0,3.150560e+03,6.351686e+03,1.000000e+00,9.190000e+02,1.600000e+03,3.085000e+03,4.969840e+05
days_on_market,589103.0,5.955989e+02,6.808799e+02,0.000000e+00,1.820000e+02,3.650000e+02,8.200000e+02,1.150600e+04
actual_vacancy,589103.0,8.752323e-01,3.304556e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
lease_term_inmonths,1067141.0,4.800054e+01,3.982621e+01,-3.950000e+02,2.400000e+01,3.600000e+01,6.000000e+01,1.236000e+03
rate_actual,219743.0,2.238252e+01,3.506006e+02,0.000000e+00,1.200000e+01,1.620000e+01,2.200000e+01,8.400000e+04
estimated_rent,928206.0,2.274745e+01,1.093183e+01,1.060077e+00,1.626206e+01,2.019501e+01,2.584223e+01,1.222861e+03


In [103]:
lease_merged.describe(include=[object]).T

,count,unique,top,freq
from_date,1067141,10835,2000-01-01,3029
to_date,794443,9332,2005-04-12,71343
lease_sign_date,998358,11553,2010-06-01,1745
lease_expiration_date,1067141,8664,2007-12-31,3483
researchmarket_name,921617,139,Los Angeles,65164
cbsa_cities,1065030,847,New York-Jersey City-White Plains,76299
cbsa_states,1065030,104,CA,111874
submarket_name,900083,2556,CBD,26843
